In [1]:
%run "../code/translator.py"
%run "../code/validation.py"

In [2]:
import re
import pickle
import numpy as np
import pandas as pd
from itertools import chain, permutations
from collections import Counter
import string
import math
import random
import matplotlib.pyplot as plt
import operator
%matplotlib inline

In [3]:
character_set = string.ascii_lowercase

In [4]:
def count_dict_to_freq_list(input_dict):
    # Fill a new dict with only and all the characters in our character set
    new_dict = {}
    for c in character_set:
        if c in input_dict.keys():
            new_dict[c] = input_dict[c]
        else:
            new_dict[c] = 0
            
    # Create a Dataframe from this dict and calculate the frequency along
    letter_freq_df = pd.DataFrame.from_dict(new_dict, orient='index')
    letter_freq_df.columns = ['count']
    letter_freq_df['freq'] = letter_freq_df['count'] / letter_freq_df['count'].sum()
    
    return letter_freq_df.sort_values('freq', ascending=False)

In [5]:
def process_word(original_word):
    return original_word.lower()

def is_valid_word(word):
    # Check if letters in character_set
    valid_word = True
    for letter in list(word):
        if letter not in character_set:
            valid_word = False
            
    return valid_word

In [6]:
import unittest

class Testjes(unittest.TestCase):

    def test_process_word(self):
        test_word = "Nederland"
        self.assertEqual('nederland', process_word(test_word))
        
    def test_check_valid_word(self):
        self.assertTrue(is_valid_word('nederland'))
        
suite = unittest.TestLoader().loadTestsFromModule(Testjes())
unittest.TextTestRunner().run(suite)

..
----------------------------------------------------------------------
Ran 2 tests in 0.001s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

In [7]:
def get_key_for_word(word):
    # Determine duplicates
    duplicate_indices = []
    for letter, letter_count in Counter(word).items():
        if letter_count > 1:
            indices = []
            # Find indices of letter
            from_index = 0
            while len(indices) < letter_count:
                new_index = word.index(letter, from_index)
                indices.append(new_index)
                from_index = new_index + 1

            # Add to duplicate indices list
            duplicate_indices.append(tuple(indices))
    duplicate_indices = tuple(sorted(duplicate_indices))
    
    return (len(word),len(Counter(word)),duplicate_indices)

def get_words_per_letter_count(language):
    """
    Get the character frequency for a given language
    """
    if language not in ['en', 'nl']:
        raise NotImplementedError(
            'Language {} not supported'.format(language))
    
    words = pickle.load(
        open("../data/{}wiktionary.p".format(language), 
             "rb"))
    
    word_dict = dict()
    
    for word in words:
        processed_word = process_word(word)
        if is_valid_word(processed_word):
            try:
                word_dict[get_key_for_word(processed_word)].append(processed_word)
            except KeyError:
                word_dict[get_key_for_word(processed_word)] = [processed_word]
        
    return word_dict

In [8]:
nl_words_per_count = get_words_per_letter_count('nl')

# Test message letter frequency

In [39]:
msg = '''
this is a test message to see if everything is working properly
we should be able to see if we can encrypt this one
english is a hard language to comprehend in such a way as it is written now
nobody really knows how many more lines we have to type before this damn thing
finally translates to something useful
let us just keep on trying
'''
msg = '''
dit is een test berichtje om te kijken of alles een beetje werkt enzo
misschien is dit nog niet voldoende maar iets is nog altijd beter dan niets toch
'''
# msg = '''
# dit is een test bericht om te kijken of het ontcijferen van een soortgelijk bericht
# toepasselijk is in de nederland taal 
# contentmanagementsystemen frituurpannen ijdelheid familietrekjes betrekkelijk gezelligheid
# '''
#msg = '''contentmanagementsystemen'''

# Remove linebreaks
msg = msg.replace('\n', ' ').replace('\r', '')

#Encode to something
random_shuffle_characters = random.sample(character_set, len(character_set)) 
cipher_enc = create_cipher(character_set, random_shuffle_characters)
msg_enc = encipher_text(msg, cipher_enc)

#Print encoding
print(msg_enc)

 pdk dg nnt kngk jnrdxckwn yl kn adwant yb hqqng nnt jnnkwn inrak ntfy ldggxcdnt dg pdk tyz tdnk syqpyntpn lhhr dnkg dg tyz hqkdwp jnknr pht tdnkg kyxc 


In [40]:
# Test the actual cipher for decoding
decipher_text(msg_enc, cipher_enc)

' dit is een test berichtje om te kijken of alles een beetje werkt enzo misschien is dit nog niet voldoende maar iets is nog altijd beter dan niets toch '

# Find actual cipher

Cycle through each group of words one by one, decipher them and see if it solves the puzzle. If not, try this with a different group first.

In [41]:
def split_sentence(sentence, split_on=' '):
    return re.findall(r"[\w']+", sentence)

In [42]:
word_enc_list = set(split_sentence(msg_enc))

In [43]:
word_enc_possibility_dict = dict()

# Run through each unique encoded word to find possibilities
for word_enc in word_enc_list:
    possibilities = nl_words_per_count[get_key_for_word(word_enc)]
    word_enc_possibility_dict[word_enc] = possibilities

In [44]:
def get_possible_words(word_enc, cipher={}):
    possible_words = word_enc_possibility_dict[word_enc]
    cipher_copy = cipher.copy()
    
    # Fill cipher with missing keys
    for c in list(character_set):
        if c not in cipher_copy.keys():
            cipher_copy[c] = '.'
    
    regex_string = decipher_text(word_enc, cipher_copy)

    regex = re.compile(regex_string)

    # Filter out possibilities with current cipher keys
    possible_words = list(filter(regex.search, possible_words))
    return possible_words
    
    # Filter out possibilities with current cipher values
    #return [pw for pw in possible_words for l in list(pw) if l not in cipher.values()]

In [45]:
def run_trough_possible_words(word_enc, cipher={}):  
    possible_words = get_possible_words(word_enc, cipher)
    
    for word_dec in possible_words:
        cipher_for_this = create_cipher(word_enc, word_dec)
        yield word_dec, {**cipher_for_this, **cipher}

In [46]:
def get_best_cipher_and_words_correct(ordered_list_words_enc, cipher_so_far={}):
    if len(ordered_list_words_enc) <= 0:
        return cipher_so_far, 0
    else:
        best_cipher=None
        best_score=0
        for possible_word_dec, cipher_used in run_trough_possible_words(ordered_list_words_enc[0], cipher_so_far):
            new_cipher_so_far, num_correct = get_best_cipher_and_words_correct(ordered_list_words_enc[1:], cipher_used)
            if num_correct+1 > best_score:
                best_cipher = new_cipher_so_far
                best_score = num_correct+1
        
        if best_score <= 0:
            # We didn't find anything, ignore this word from now on
            return get_best_cipher_and_words_correct(ordered_list_words_enc[1:], cipher_so_far)
        else:
            return best_cipher, best_score
    
def get_good_enough_cipher_and_words_correct(ordered_list_words_enc, cipher_so_far={}):
    if len(ordered_list_words_enc) <= 0:
        return cipher_so_far, 0
    else:
        for possible_word_dec, cipher_used in run_trough_possible_words(ordered_list_words_enc[0], cipher_so_far):
            new_cipher_so_far, num_correct = get_best_cipher_and_words_correct(ordered_list_words_enc[1:], cipher_used)
            if (num_correct+1) / (len(ordered_list_words_enc)) > 0.8:
                return new_cipher_so_far, num_correct+1
        
        # We didn't find anything, ignore this word from now on
        return get_best_cipher_and_words_correct(ordered_list_words_enc[1:], cipher_so_far)

# New plan, try bootstrapping

In [47]:
def calc_complexity(word_list, cipher):
    complexity = 1
    for w in word_list:
        complexity *= len(get_possible_words(w,cipher))
    
    return complexity

In [71]:
def calc_predictive_value(word_list):
    amount_of_word_occurences = 4
    
    for w in word_list:
        possibilities = calc
    
    return word_list

calc_predictive_value(['syqpyntpn', 'pdk', 'pht', 'hqkdwp'])

['uiteinden', 'voldoende', 'rabiaseis', 'cariaseis', 'onmensjes', 'vaciaseis', 'radiaseis', 'variaseis']
['soi', 'yuv', 'god', 'usa', 'anu', 'nai', 'cie', 'rof', 'keu', 'gar', 'kum', 'bat', 'aer', 'hbq', 'ahi', 'flm', 'hit', 'iwu', 'abq', 'cop', 'nuk', 'aug', 'rie', 'aje', 'ord', 'mig', 'jam', 'hiv', 'cep', 'lim', 'com', 'meg', 'isu', 'gol', 'yan', 'ona', 'bio', 'wal', 'hja', 'neb', 'qha', 'asv', 'ins', 'kis', 'bie', 'jim', 'kaj', 'tok', 'hun', 'klm', 'puc', 'kyr', 'gis', 'afe', 'ule', 'gif', 'nox', 'nda', 'one', 'mus', 'ler', 'pie', 'vin', 'jut', 'bei', 'yar', 'hwb', 'riv', 'jsf', 'lag', 'sep', 'elv', 'sly', 'fix', 'yoz', 'big', 'fel', 'saz', 'pib', 'bao', 'fay', 'yas', 'iar', 'you', 'rum', 'cin', 'bra', 'cas', 'vwo', 'dek', 'oir', 'gae', 'evn', 'age', 'bas', 'hlm', 'sie', 'mcv', 'dob', 'ima', 'jom', 'way', 'fun', 'wan', 'jeb', 'dit', 'eds', 'sop', 'uws', 'atr', 'okt', 'bwo', 'nal', 'uda', 'omi', 'fcd', 'lai', 'deu', 'lon', 'doc', 'pst', 'nor', 'shf', 'mdi', 'abw', 'guy', 'tae', 'kir'

['syqpyntpn', 'pdk', 'pht', 'hqkdwp']

In [93]:
def generate_subset_words_per_letter(word_list, cipher_fixed, max_complexity=1e11, min_set_size=4):
    # Determine the best order to walk through the letters (certain onces will be easier to workout)
    # Now we choose the letters that occur in most words first
    letter_scores = list()
    for char in list(character_set):
        word_set = [w for w in word_list if char in w]
        if len(word_set) == 0:
            avg_word_set_complexity = 0
        else:
            avg_word_set_complexity = min([calc_complexity([w],{}) for w in word_set])
        score = len(word_set) - avg_word_set_complexity
        letter_scores.append((char, score, word_set))
    
    letter_scores = sorted(letter_scores, key=lambda x: x[1], reverse=True)

    for char, char_score, word_set in letter_scores:
        complexity = calc_complexity(word_set, cipher_fixed)

        # As long as the complexity is too high, reduce the set size
        while complexity > max_complexity and len(word_set) >= min_set_size:
            word_set = word_set[:-1]
            complexity = calc_complexity(word_set, cipher_fixed)

        # We only take sets of minimum x words
        if len(word_set) >= min_set_size:
            yield char, word_set

In [94]:
# Sort the word_encoded list based on possibilities
word_enc_list_ordered = sorted(word_enc_possibility_dict, key=lambda k: len(word_enc_possibility_dict[k]), reverse=False)

cipher_already_fix = {}
min_set_size = 10
while min_set_size > 0:
    print("---- Start deciphering with minimum set of {}".format(min_set_size))
    
    gen_subsets = generate_subset_words_per_letter(word_enc_list_ordered, cipher_already_fix, max_complexity=1e12, min_set_size=min_set_size)
    for used_letter, used_set in gen_subsets:
        if used_letter not in cipher_already_fix.keys():
            print("-- Letter {}".format(used_letter))
            found_cipher, correct_words = get_best_cipher_and_words_correct(used_set,cipher_already_fix)
            correctly_translated = correct_words/len(used_set)

            try:
                if found_cipher[used_letter] in cipher_already_fix.values():
                    print("!!!! PROBLEM: this letter has been used to translate to already")
                else:
                    cipher_already_fix[used_letter] = found_cipher[used_letter]

                    print("\tDecided for {0}:{1}".format(used_letter, found_cipher[used_letter]))
                    print("\tNumber of correctly translated words: {0}/{1} ({2}%)".format(correct_words, len(used_set), int(100*correctly_translated)))
            except KeyError:
                print("We'd hoped to find this one, but couldnt: {}".format(used_letter))

    min_set_size -= 1
    
print()
print("Number of keys found: {}".format(len(cipher_already_fix)))
print(cipher_already_fix)

---- Start deciphering with minimum set of 10
---- Start deciphering with minimum set of 9
---- Start deciphering with minimum set of 8
---- Start deciphering with minimum set of 7
---- Start deciphering with minimum set of 6
-- Letter n
	Decided for n:e
	Number of correctly translated words: 6/6 (100%)
-- Letter t
	Decided for t:n
	Number of correctly translated words: 6/6 (100%)
-- Letter d
	Decided for d:i
	Number of correctly translated words: 6/6 (100%)
-- Letter g
	Decided for g:s
	Number of correctly translated words: 6/6 (100%)
-- Letter k
	Decided for k:t
	Number of correctly translated words: 6/6 (100%)
---- Start deciphering with minimum set of 5
-- Letter y
	Decided for y:o
	Number of correctly translated words: 5/6 (83%)
---- Start deciphering with minimum set of 4
-- Letter p
	Decided for p:d
	Number of correctly translated words: 4/4 (100%)
-- Letter w
	Decided for w:j
	Number of correctly translated words: 4/4 (100%)
-- Letter h
	Decided for h:a
	Number of correctly tra

In [95]:
found_cipher_copy = cipher_already_fix.copy()

# Figure out which values are missing

# Fill cipher with missing keys
for c in list(character_set):
    if c not in found_cipher_copy.keys():
        found_cipher_copy[c] = '_'

print(decipher_text(msg_enc, found_cipher_copy))
msg_enc

 dit is een test berichtje om te pijpen o_ alles een beetje werpt en_o misschien is dit nox niet voldoende maar iets is nox altijd beter dan niets toch 


' pdk dg nnt kngk jnrdxckwn yl kn adwant yb hqqng nnt jnnkwn inrak ntfy ldggxcdnt dg pdk tyz tdnk syqpyntpn lhhr dnkg dg tyz hqkdwp jnknr pht tdnkg kyxc '